In [21]:
import pandas as pd
import numpy as np

# จำลองข้อมูลที่ไม่มีความเป็นระเบียบ (Messy Data)
data = {
    'Asset_ID': ['SV-001', 'SV-002', 'sv-003', np.nan, 'SV-005', 'SV-001', 'SV-007'], # มีค่าว่าง, พิมพ์เล็กใหญ่ไม่เท่ากัน, ซ้ำ
    'Purchase_Date': ['2020-01-15', '15/02/2020', 'March 5, 2021', '2022-12-10', 'N/A', '2020-01-15', '2023-06-01'], # รูปแบบวันที่มั่ว, มี text ปน
    'Cost (THB)': ['25,000', '32000', '18,500.50', '50000', 'Free', '25,000', '-'], # มี comma, ไม่มี comma, มี text, มีขีด
    'Ram_GB': [16, 32, '16GB', 64, 8, 16, '32 GB'], # ปนกันระหว่าง int และ string มีหน่วยติดมา
    'Status': ['Active', 'active ', 'Maintenace', 'Down', 'Active', 'Active', 'Retired'], # มีวรรคเกิน (trailing space), สะกดผิด (Maintenance)
    'IP_Address': ['192.168.1.1', '192.168.1.2', '10.0.0.5', '192.168.1.20', 'No IP', '192.168.1.1', '172.16.0.1']
}

df_raw = pd.DataFrame(data)

print("โจทย์ของป๋าพร้อมแล้ว! หน้าตาข้อมูลดิบ:")
print(df_raw)


โจทย์ของป๋าพร้อมแล้ว! หน้าตาข้อมูลดิบ:
  Asset_ID  Purchase_Date Cost (THB) Ram_GB      Status    IP_Address
0   SV-001     2020-01-15     25,000     16      Active   192.168.1.1
1   SV-002     15/02/2020      32000     32     active    192.168.1.2
2   sv-003  March 5, 2021  18,500.50   16GB  Maintenace      10.0.0.5
3      NaN     2022-12-10      50000     64        Down  192.168.1.20
4   SV-005            N/A       Free      8      Active         No IP
5   SV-001     2020-01-15     25,000     16      Active   192.168.1.1
6   SV-007     2023-06-01          -  32 GB     Retired    172.16.0.1


In [ ]:
df = df_raw.copy()

1. Standardize Text (จัดการ Asset_ID)

โจทย์: แปลงเป็นตัวพิมพ์ใหญ่ และลบแถวที่เป็นค่าว่าง

In [7]:
# 1. ลบแถวที่ Asset_ID เป็น NaN ค่าว่าง ก่อน
# subset=['Asset_ID'] บอกว่าให้เช็คเฉพาะคอลัมน์นี้นะ ถ้าคอลัมน์อื่นว่างช่างมัน
df.dropna(subset=['Asset_ID'], inplace=True)

# 2. แปลงเป็นตัวใหญ่
# .str คือเรียกฟังก์ชั่นการจัดการข้อความ, .upper() ทำตัวใหญ่ทั้งหมด
df['Asset_ID'] = df['Asset_ID'].str.upper()


print('ข้อ 1 เสร็จสิ้น')
print(df['Asset_ID'])


ข้อ 1 เสร็จสิ้น
0    SV-001
1    SV-002
2    SV-003
4    SV-005
5    SV-001
6    SV-007
Name: Asset_ID, dtype: object


2. Remove Duplicates (ลบข้อมูลซ้ำ)

โจทย์: ลบแถวที่ซ้ำกันทิ้ง

In [10]:
# keep='first' เก็บตัวแรกที่เจอ ลบตัวหลังทิ้ง
# inplace=True คือแก้ทับลงในตัวแปรเดิม df เลย ไม่ต้องสร้างตัวแปรใหม่มารับ
df.drop_duplicates(inplace=True)

print('\nข้อ 2 เสร็จสิ้น:')
print(df['Asset_ID'])


ข้อ 2 เสร็จสิ้น:
0    SV-001
1    SV-002
2    SV-003
4    SV-005
6    SV-007
Name: Asset_ID, dtype: object


3. Cleaning Numbers (ล้างหน่วยออกจาก RAM)

โจทย์: เอาคำว่า "GB" ออก และแปลงเป็น Integer จุดตาย: ข้อมูลดิบมีทั้งตัวเลข (16) และตัวหนังสือ ("16GB") 

ปนกัน ถ้าสั่ง .str.replace เลยจะ Error เพราะเลขใช้ฟังก์ชั่นสตริงไม่ได้

In [11]:
# 1. .astype(str) -> แปรทุกอย่างให้เป็นตัวหนังสือ
# 2. .str.replace('GB', '') เอาคำว่า GB ออก
# 3. .str.strip() ตัดช่องว่างหน้าหลัง
# 4. .astype(int) สุดท้าย แปรข้อมูลเป็น ตัวเลขจำนวนเต็ม
df['Ram_GB'] = df['Ram_GB'].astype(str).str.replace('GB', '').str.strip().astype(int)

print('\nข้อ 3 เสร็จสิ้น (Ram เป็นตัวเลขแล้ว คำนวณได้): ')
print(df['Ram_GB'].dtype) # เช็คชนิดข้อมูล


ข้อ 3 เสร็จสิ้น (Ram เป็นตัวเลขแล้ว คำนวณได้): 
int64


In [12]:
df

,Asset_ID,Purchase_Date,Cost (THB),Ram_GB,Status,IP_Address
0,SV-001,2020-01-15,"25,000",16,Active,192.168.1.1
1,SV-002,15/02/2020,32000,32,active,192.168.1.2
2,SV-003,"March 5, 2021","18,500.50",16,Maintenace,10.0.0.5
4,SV-005,N/A,Free,8,Active,No IP
6,SV-007,2023-06-01,-,32,Retired,172.16.0.1


4. Handling Dirty Currency (จัดการเรื่องเงิน)

โจทย์: จัดการ "Free", "-", "," และแปลงเป็น Float

In [14]:
# 1. จัดการเคสพิเศษก่อน (free, -) ให้เป็น 0
# ใช้ replace แบบใส่ Dictionary {คำเดิม: คำใหม่}
df['Cost (THB)'] = df['Cost (THB)'].replace({'Free':'0', '-': '0'})

# 2. เอาเครื่องหมายคอมม่าออก , 
# ต้องแปรเป็น str ก่อนแล้วค่อย replace
df['Cost (THB)'] = df['Cost (THB)'].astype(str).str.replace(',', '')

# แปลงข้อมูลเป็น เลขทศนิยม
df['Cost (THB)'] = df['Cost (THB)'].astype(float)

print('\nข้อ 4 เสร็จสิ้น')
print(df['Cost (THB)'])


ข้อ 4 เสร็จสิ้น
0    25000.0
1    32000.0
2    18500.5
4        0.0
6        0.0
Name: Cost (THB), dtype: float64


In [23]:
df_raw

,Asset_ID,Purchase_Date,Cost (THB),Ram_GB,Status,IP_Address
0,SV-001,2020-01-15,"25,000",16,Active,192.168.1.1
1,SV-002,15/02/2020,32000,32,active,192.168.1.2
2,sv-003,"March 5, 2021","18,500.50",16GB,Maintenace,10.0.0.5
3,NaN,2022-12-10,50000,64,Down,192.168.1.20
4,SV-005,N/A,Free,8,Active,No IP
5,SV-001,2020-01-15,"25,000",16,Active,192.168.1.1
6,SV-007,2023-06-01,-,32 GB,Retired,172.16.0.1


5. Fixing Dates (แปลงวันที่)

โจทย์: แปลงเป็น datetime object จุดตาย: errors='coerce' 

คือท่าไม้ตาย ถ้าแปลงไม่ได้ (เช่น N/A) อย่า Error แต่ให้เปลี่ยนเป็นค่า NaT (Not a Time) แทน

In [ ]:
df['Purchase_Date'] = pd.to_datetime(
    df['Purchase_Date'],
    dayfirst=True,
    format='maxed',
    errors='coerce' )
print('\nข้อ 5 เสร็จสิ้น')

,Asset_ID,Purchase_Date,Cost (THB),Ram_GB,Status,IP_Address
0,SV-001,2020-01-15,25000.0,16,Active,192.168.1.1
1,SV-002,NaT,32000.0,32,active,192.168.1.2
2,SV-003,NaT,18500.5,16,Maintenace,10.0.0.5
4,SV-005,NaT,0.0,8,Active,No IP
6,SV-007,2023-06-01,0.0,32,Retired,172.16.0.1
